In [1]:
import torch
import torch.nn as nn

In [2]:
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()             
        self.linear1 = nn.Linear(6, 3)
        self.linear2 = nn.Linear(3, 1)
        
    def forward(self, x):   
        x = self.linear1(x)
        output = self.linear2(x)
        return output

In [3]:
ffnn = FFNN()